<a href="https://colab.research.google.com/github/ferranfont/Introduccion_al_trading_algoritmico/blob/main/Quant_momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**QUANTITATIVE MOMENTUM SYSTEM**

In [87]:
import pandas as pd
import numpy as np
import requests
import matplotlib
import math
from scipy import stats # Esta libreria es nueva y la usaremos para calcular el percentil

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
# Importamos todos los tickets del SP500
stocks = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [62]:
# Hacemos un API a un end point de momentum
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol='BA'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avg10Volume': 8105951,
 'avg30Volume': 10117783,
 'beta': 1.368715126130896,
 'companyName': 'Boeing Co.',
 'day200MovingAvg': 218.1,
 'day30ChangePercent': -0.024221604590951195,
 'day50MovingAvg': 213.12,
 'day5ChangePercent': -0.015795783970610662,
 'dividendYield': 0,
 'employees': 141174,
 'exDividendDate': '',
 'float': 0,
 'marketcap': 123688806906,
 'maxChangePercent': 2.173827479393077,
 'month1ChangePercent': -0.03826036842015353,
 'month3ChangePercent': -0.02498946901024187,
 'month6ChangePercent': -0.015100972091536389,
 'nextDividendDate': '',
 'nextEarningsDate': '',
 'peRatio': -14.692411325317401,
 'sharesOutstanding': 583910799,
 'ttmDividendRate': 0,
 'ttmEPS': -14.8,
 'week52change': -0.03949954682473582,
 'week52high': 282.82,
 'week52highSplitAdjustOnly': 288.41,
 'week52low': 187.57,
 'week52lowSplitAdjustOnly': 186.68,
 'year1ChangePercent': -0.04002103432688062,
 'year2ChangePercent': -0.3907600204887263,
 'year5ChangePercent': 0.303422384281532,
 'ytdChangePe

In [63]:
# Consultamos un campo en particular
data['month1ChangePercent']*100

-3.826036842015353

In [64]:
round(data['peRatio'],1)

-14.7

In [65]:
my_colums = ['Ticker','Descripción', 'Rentab_1año','Rentab_6m','Rentab_1m','PER','n_acciones_a_comprar']
final_dataframe = pd.DataFrame(columns=my_colums)
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar


In [66]:
final_dataframe = final_dataframe.append(
                                        pd.Series([i, 
                                                   data['companyName'], 
                                                   round(data['year1ChangePercent'],2),
                                                   round(data['month6ChangePercent'],2),
                                                   round(data['month1ChangePercent'],2),
                                                   round(data['peRatio'],1),
                                                   'N/A'],  
                                                  index =my_colums), 
                                        ignore_index = True)
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,CAT,Boeing Co.,-0.04,-0.02,-0.04,-14.7,N/A


In [67]:
# Rellenamos el dataframe en cada una de sus filas (stocks tickers) con el valor que obtenemos de la API call
for i in stocks['Ticker'][:80]: #para no ralentizar el proceso lo haremos tan sólo para las primeros 80 símbolos [:80]
  api_url = f'https://sandbox.iexapis.com/stable/stock/{i}/stats?token=Tpk_059b97af715d417d9f49f50b51b1c448'
  data = requests.get(api_url).json()
  final_dataframe = final_dataframe.append(
                                        pd.Series([i, 
                                                   data['companyName'], 
                                                   round(data['year1ChangePercent'],2),
                                                   round(data['month6ChangePercent'],2),
                                                   round(data['month1ChangePercent'],2),
                                                   round(data['peRatio'],1),
                                                   'N/A'], 
                                                  index =my_colums), 
                                        ignore_index = True)

In [68]:
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,CAT,Boeing Co.,-0.04,-0.02,-0.04,-14.7,N/A
1,A,Agilent Technologies Inc.,0.05,-0.22,-0.06,16.5,N/A
2,AAL,American Airlines Group Inc,-0.04,-0.05,0.03,-3.7,N/A
3,AAP,Advance Auto Parts Inc,0.35,0.06,-0.07,10.9,N/A
4,AAPL,Apple Inc,0.31,0.15,0.01,27.9,N/A
...,...,...,...,...,...,...,...
76,C,Citigroup Inc,0.00,-0.07,0.01,6.1,N/A
77,CAG,Conagra Brands Inc,0.09,0.09,0.01,16.3,N/A
78,CAH,"Cardinal Health, Inc.",0.08,0.08,0.04,27.5,N/A
79,CARR,Carrier Global Corp,0.19,-0.21,-0.10,11.5,N/A


In [69]:
# Vamos a retirar los que tienen bajo momentum con el método .sort_value()
final_dataframe.sort_values('Rentab_1año', ascending=False, inplace=True)
final_dataframe.reset_index(inplace=True) #reseteamos el índice de nuevo
final_dataframe[:30]


,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,43,APA,APA Corporation,0.66,1.04,-0.00,20.4,N/A
1,38,ANET,Arista Networks Inc,0.63,0.38,0.00,23.9,N/A
2,55,AZO,Autozone Inc.,0.61,0.18,-0.01,17.4,N/A
3,40,ANTM,Anthem Inc,0.54,0.20,-0.00,9.0,N/A
4,54,AXP,American Express Co.,0.50,0.23,0.21,9.5,N/A
5,19,AIG,American International Group Inc,0.50,0.19,0.05,2.9,N/A
6,20,AIV,Apartment Investment & Management Co.,0.47,0.09,-0.01,-36.1,N/A
7,5,ABBV,Abbvie Inc,0.45,0.26,0.07,35.4,N/A
8,12,ADM,Archer Daniels Midland Co.,0.40,0.31,0.09,8.3,N/A
9,65,BK,Bank Of New York Mellon Corp,0.38,0.08,-0.04,14.2,N/A


In [74]:
final_dataframe['Percentil_1año']=0
final_dataframe['Percentil_6meses']=0
final_dataframe['Percentil_1mes']=0
final_dataframe

,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes
0,43,APA,APA Corporation,0.66,1.04,-0.00,20.4,N/A,0,0,0
1,38,ANET,Arista Networks Inc,0.63,0.38,0.00,23.9,N/A,0,0,0
2,55,AZO,Autozone Inc.,0.61,0.18,-0.01,17.4,N/A,0,0,0
3,40,ANTM,Anthem Inc,0.54,0.20,-0.00,9.0,N/A,0,0,0
4,54,AXP,American Express Co.,0.50,0.23,0.21,9.5,N/A,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
76,49,ATVI,Activision Blizzard Inc,-0.19,-0.03,-0.01,24.9,N/A,0,0,0
77,17,AES,AES Corp.,-0.23,-0.09,-0.07,26.6,N/A,0,0,0
78,39,ANSS,Ansys Inc.,-0.25,-0.17,-0.11,58.2,N/A,0,0,0
79,63,BIIB,Biogen Inc,-0.25,-0.38,-0.09,10.4,N/A,0,0,0


In [98]:
n_valores = 30

In [103]:
for i in range(0,n_valores):
  final_dataframe.loc[i,'Percentil_1año'] = round(stats.percentileofscore(final_dataframe['Rentab_1año'],final_dataframe.loc[i,'Rentab_1año']),1)
  final_dataframe.loc[i,'Percentil_6meses'] = round(stats.percentileofscore(final_dataframe['Rentab_6m'],final_dataframe.loc[i,'Rentab_6m']),1)
  final_dataframe.loc[i,'Percentil_1mes'] = round(stats.percentileofscore(final_dataframe['Rentab_1m'],final_dataframe.loc[i,'Rentab_1m']),1)
final_dataframe[:30]

,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes
0,43.0,APA,APA Corporation,0.66,1.04,-0.00,20.4,N/A,88.0,88.0,60.9
1,38.0,ANET,Arista Networks Inc,0.63,0.38,0.00,23.9,N/A,87.0,85.9,60.9
2,55.0,AZO,Autozone Inc.,0.61,0.18,-0.01,17.4,N/A,85.9,76.1,52.7
3,40.0,ANTM,Anthem Inc,0.54,0.20,-0.00,9.0,N/A,84.8,78.8,60.9
4,54.0,AXP,American Express Co.,0.50,0.23,0.21,9.5,N/A,83.2,80.4,88.0
5,19.0,AIG,American International Group Inc,0.50,0.19,0.05,2.9,N/A,83.2,77.2,78.8
6,20.0,AIV,Apartment Investment & Management Co.,0.47,0.09,-0.01,-36.1,N/A,81.5,64.1,52.7
7,5.0,ABBV,Abbvie Inc,0.45,0.26,0.07,35.4,N/A,80.4,82.6,81.0
8,12.0,ADM,Archer Daniels Midland Co.,0.40,0.31,0.09,8.3,N/A,79.3,84.8,83.7
9,65.0,BK,Bank Of New York Mellon Corp,0.38,0.08,-0.04,14.2,N/A,78.3,60.3,35.9
